In [1]:
%load_ext kedro.ipython

[06/22/24 11:16:51] INFO     Registered line magic '%reload_kedro'                                   ]8;id=125778;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=213465;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\ipython\__init__.py#58\58]8;;\

[06/22/24 11:16:52] INFO     Registered line magic '%load_node'                                      ]8;id=339253;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=420260;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\ipython\__init__.py#60\60]8;;\

                    INFO     Resolved project path as:                                              ]8;id=758111;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=257863;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\ipython\__init__.py#171\171]8;;\
                             C:\Users\dggua\Downloads\MLOPS\project\MLOPS\project-mlops.                           
                             To set a different path, run '%reload_kedro <project_root>'                           

[06/22/24 11:16:56] INFO     Registering new custom resolver: 'km.random_name'                    ]8;id=138123;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py\mlflow_hook.py]8;;\:]8;id=822661;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro_mlflow\framework\hooks\mlflow_hook.py#65\65]8;;\

                    INFO     The 'tracking_uri' key in mlflow.yml is relative            ]8;id=976116;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro_mlflow\config\kedro_mlflow_config.py\kedro_mlflow_config.py]8;;\:]8;id=4320;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro_mlflow\config\kedro_mlflow_config.py#260\260]8;;\
                             ('server.mlflow_(tracking|registry)_uri = mlruns'). It is                             
                             converted to a valid uri:                                                             
                             'file:///C:/Users/dggua/Downloads/MLOPS/project/MLOPS/proje                           
                             ct-mlops/mlruns'                                                                      

                    INFO     Kedro project project-mlops                                            ]8;id=797349;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=638648;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\ipython\__init__.py#141\141]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=726852;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=467737;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\ipython\__init__.py#142\142]8;;\
                             'pipelines'                                                                           

[06/22/24 11:16:58] INFO     Registered line magic 'run_viz'                                        ]8;id=906400;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\ipython\__init__.py\__init__.py]8;;\:]8;id=633469;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\ipython\__init__.py#148\148]8;;\

In [2]:
df = catalog.load("credits")

[06/22/24 11:17:16] INFO     Loading data from credits (CSVDataset)...                          ]8;id=433862;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=779243;file://C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\kedro\io\data_catalog.py#508\508]8;;\

In [6]:
import logging
from typing import Any, Dict, Tuple

import numpy as np
import pandas as pd

from great_expectations.core import ExpectationSuite, ExpectationConfiguration

from pathlib import Path

from kedro.config import OmegaConfigLoader
from kedro.framework.project import settings
import hopsworks

In [11]:
categorical_features = df.select_dtypes(include=['object']).columns
numerical_f = df.select_dtypes(include=['float64']).columns
categorical_f = categorical_features[:-1]

In [26]:
def build_expectations(name, type, columns):

    value = {'checking_status' : ['<0' '0<=X<200' 'no checking' '>=200'],
     'credit_history': ['critical/other existing credit' 'existing paid' 'delayed previously'
 'no credits/all paid' 'all paid'],
     'purpose':['radio/tv' 'education' 'furniture/equipment' 'new car' 'used car'
 'business' 'domestic appliance' 'repairs' 'other' 'retraining'],
     'savings_status':['no known savings' '<100' '500<=X<1000' '>=1000' '100<=X<500'],
     'employment':['>=7' '1<=X<4' '4<=X<7' 'unemployed' '<1'],
     'personal_status':['male single' 'female div/dep/mar' 'male div/sep' 'male mar/wid'],
     'other_parties':['none' 'guarantor' 'co applicant'],
    'property_magnitude':['real estate' 'life insurance' 'no known property' 'car'],
    'other_payment_plans':['none' 'bank' 'stores'],
    'housing':['own' 'for free' 'rent'],
    'job':['skilled' 'unskilled resident' 'high qualif/self emp/mgmt'
 'unemp/unskilled non res'],
    'own_telephone':['yes' 'none'],
    'foreign_worker':['yes' 'no']   
    }

    
    expectation_suite_bank = ExpectationSuite(
        expectation_suite_name= name
    )
    
    if type == 'numerical':
        columns = ['duration', 'credit_amount', 'installment_commitment',
       'residence_since', 'age', 'existing_credits', 'num_dependents']
        for i in columns:
            expectation_suite_bank.add_expectation(
                        ExpectationConfiguration(
                            expectation_type="expect_column_values_to_be_of_type",
                            kwargs={"column": i, "type_": "float64"},
                        )
                    )
    elif type == 'categorical':
        for i in columns:
            expectation_suite_bank.add_expectation(
                        ExpectationConfiguration(
                            expectation_type="expect_column_distinct_values_to_be_in_set",
                            kwargs={"column": i, "value_set": value[i]},
                        )
                    )
    else:
        print(type)
        expectation_suite_bank.add_expectation(
                        ExpectationConfiguration(
                            expectation_type="expect_column_distinct_values_to_be_in_set",
                            kwargs={"column": 'class', "value_set": ['good' 'bad']},
                        )
                    )
    return expectation_suite_bank

In [41]:
validation_expectation_numerical = build_expectations("numerical_expectation","numerical", numerical_f)
validation_expectation_categorical = build_expectations("categorical_expectations","categorical",categorical_f)
validation_expectation_target = build_expectations("target_expectations","target",'')

target


In [79]:
numerical_feature_descriptions =[]
categorical_feature_descriptions =[]
target_feature_descriptions =[]
df_numeric = df[numerical_features]
df_categorical = df[categorical_features]
df_target = df['class']


In [84]:
df_numeric.loc[:, "index"] = df_numeric.index
df_categorical.loc[:, "index"] = df_categorical.index
df_target.loc["index"] = df_target.index


In [93]:
def save_feature_store(df, group_name, expectations,feature_group_version,description, ):
    print(df.index)
    project = hopsworks.login(
        api_key_value="SdmqGu8BDdngCQsZ.b1fUsORi1IuRzIKsu7sEKPfcVk3wNxiZk7wL7tKUlbS5BHJzwrsf82e8idCdUcNU", project="mlopsDALI"
    )
    feature_store = project.get_feature_store()

    object_feature_group = feature_store.get_or_create_feature_group(
        name=group_name,
        version=feature_group_version,
        description= description,
        primary_key=["index"],
        online_enabled=False,
        expectation_suite=expectations,
    )
    object_feature_group.insert(
        features=df,
        overwrite=False,
        write_options={
            "wait_for_job": True,
        },
    )
    object_feature_group.statistics_config = {
        "enabled": True,
        "histograms": True,
        "correlations": True,
    }
    object_feature_group.update_statistics_config()
    object_feature_group.compute_statistics()

    return object_feature_group


In [94]:
object_fs_categorical_features= save_feature_store(df_categorical,'categorical_features',validation_expectation_categorical,1,'categorical_feature_descriptions')
object_fs_num_features= save_feature_store(df_numeric,'numerical_features',validation_expectation_numerical,1,'numerical_feature_descriptions')


RangeIndex(start=0, stop=1000, step=1)
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/820377
Connected. Call `.close()` to terminate connection gracefully.


[06/22/24 12:24:31] WARNING  DeprecationWarning: is_sparse is deprecated and will be removed in a   ]8;id=82627;file://C:\Users\dggua\anaconda3\envs\test_env\lib\warnings.py\warnings.py]8;;\:]8;id=578856;file://C:\Users\dggua\anaconda3\envs\test_env\lib\warnings.py#109\109]8;;\
                             future version. Check `isinstance(dtype, pd.SparseDtype)` instead.                    
                                                                                                                   

Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/820377/fs/816200/fg/930132


Uploading Dataframe: 0.00% |          | Rows 0/1000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: categorical_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/820377/jobs/named/categorical_features_1_offline_fg_materialization/executions
Statistics Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/820377/jobs/named/categorical_features_1_compute_stats_22062024112621/executions
RangeIndex(start=0, stop=1000, step=1)
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/820377
Connected. Call `.close()` to terminate connection gracefully.


[06/22/24 12:28:02] WARNING  DeprecationWarning: is_sparse is deprecated and will be removed in a   ]8;id=903565;file://C:\Users\dggua\anaconda3\envs\test_env\lib\warnings.py\warnings.py]8;;\:]8;id=379201;file://C:\Users\dggua\anaconda3\envs\test_env\lib\warnings.py#109\109]8;;\
                             future version. Check `isinstance(dtype, pd.SparseDtype)` instead.                    
                                                                                                                   

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/820377/fs/816200/fg/930133
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/820377/fs/816200/fg/930133


Uploading Dataframe: 0.00% |          | Rows 0/1000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: numerical_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/820377/jobs/named/numerical_features_1_offline_fg_materialization/executions
Statistics Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/820377/jobs/named/numerical_features_1_compute_stats_22062024113031/executions
Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
           991,     992,     993,     994,     995,     996,     997,     998,
           999, 'index'],
      dtype='object', length=1001)
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/820377
Connected. Call `.close()` to terminate connection gracefully.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 object_fs_categorical_features= save_feature_store(df_categorical,'categorical_features'     │
│   2 object_fs_num_features= save_feature_store(df_numeric,'numerical_features',validation_ex     │
│ ❱ 3 object_fs_target_features= save_feature_store(df_target,'target_features',validation_exp     │
│   4                                                                                              │
│                                                                                                  │
│ in save_feature_store:16                                                                         │
│                                                                                                  │
│   13 │   │   online_enabled=False,                                                               │
│   14 │   │   expectation_suite=expectations,                                                     │
│   15 │   )                                                                                       │
│ ❱ 16 │   object_feature_group.insert(                                                            │
│   17 │   │   features=df,                                                                        │
│   18 │   │   overwrite=False,                                                                    │
│   19 │   │   write_options={                                                                     │
│                                                                                                  │
│ C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\hsfs\feature_group.py:2519 in insert    │
│                                                                                                  │
│ C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\hsfs\engine\python.py:536 in            │
│ convert_to_default_dataframe                                                                     │
│                                                                                                  │
│ C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\pandas\core\ops\common.py:76 in         │
│ new_method                                                                                       │
│                                                                                                  │
│ C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\pandas\core\arraylike.py:40 in __eq__   │
│                                                                                                  │
│ C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\pandas\core\series.py:5803 in           │
│ _cmp_method                                                                                      │
│                                                                                                  │
│ C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\pandas\core\ops\array_ops.py:346 in     │
│ comparison_op                                                                                    │
│                                                                                                  │
│ C:\Users\dggua\anaconda3\envs\test_env\lib\site-packages\pandas\core\ops\array_ops.py:131 in     │
│ comp_method_OBJECT_ARRAY                                                                         │
│                                                                                                  │
│ in pandas._libs.ops.scalar_compare:95                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: The truth value of an array with more than one el

In [112]:
df_target = pd.DataFrame()
df_target['class'] = df['class']
df_target.loc[:, "index"] = df_target.index 

In [113]:
object_fs_target_features= save_feature_store(df_target,'target_features',validation_expectation_target,1,'target_feature_descriptions')

RangeIndex(start=0, stop=1000, step=1)
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/820377
Connected. Call `.close()` to terminate connection gracefully.


[06/22/24 12:50:33] WARNING  DeprecationWarning: is_sparse is deprecated and will be removed in a   ]8;id=679514;file://C:\Users\dggua\anaconda3\envs\test_env\lib\warnings.py\warnings.py]8;;\:]8;id=74870;file://C:\Users\dggua\anaconda3\envs\test_env\lib\warnings.py#109\109]8;;\
                             future version. Check `isinstance(dtype, pd.SparseDtype)` instead.                    
                                                                                                                   

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/820377/fs/816200/fg/930134
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/820377/fs/816200/fg/930134


Uploading Dataframe: 0.00% |          | Rows 0/1000 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: target_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/820377/jobs/named/target_features_1_offline_fg_materialization/executions
Statistics Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/820377/jobs/named/target_features_1_compute_stats_22062024115230/executions
